In [59]:
import re
import string
import pickle
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer

ps = PorterStemmer()

#### Data preprocessing

In [60]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [61]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [62]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [63]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist() 

In [64]:
def preprocess_data (text):
    data = pd.DataFrame([text], columns=['tweet']) 

    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data['tweet']= data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data['tweet'] = data['tweet'].str.replace('\d+', '', regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))

    return data['tweet']

In [65]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)  # Only append after processing the sentence

    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)
    return vectorized_list_new

In [67]:
def get_prediction (vectorized_text):
    prediction = model.predict(vectorized_text)

    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [69]:
text = 'awesome product. i love it'

preprocessed_text = preprocess_data(text)
vectorized_text = vectorizer(preprocessed_text, tokens)

get_prediction(vectorized_text)

'positive'